In [1]:
!pip install pymysql > /dev/null

In [2]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [3]:
import json 
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [4]:
import pymysql 
conn = pymysql.connect(
     host = config['host'],
     user = config['user'],
     password = config['password'],
     database = config['database'],
     port = config['port'])

# 패스워드 암호화
- secure hash Algorithm (sha-256)
-base64로 인코딩

In [7]:
import hashlib
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()


b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- base 64로 인코딩하기


In [8]:
import base64

In [9]:
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [10]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('UTF-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [11]:
len(hashed_pwd)

44

-패스우드를 생성하는 함수

In [13]:
import hashlib
import base64

def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('UTF-8')
    return hashed_pwd

In [14]:
gen_pwd('qwerty')

'ZehL4zUy+3hMSBKWdfnv86aCsnFowOp0Syz1juAjN8U='

## user table에 패스워드 넣기
- uid +21

In [20]:
sql = 'select uid from users;'
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchall()
result

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('gdhong2',),
 ('jbpark',),
 ('jbpark2',),
 ('wjlee',))

In [27]:
#패스워드 넣기
sql_update = 'update users set pwd=%s where uid=%s;'
for line in result:
    uid = line[0]
    pwd = gen_pwd(uid+'21')
    cur.execute(sql_update,(pwd, uid))
conn.commit()

- email 추가 uid@mc.com


In [29]:
sql_email = 'update users set email=%s where uid=%s;'
for line in result:
    uid = line[0]
    email = uid+'@mc.com'
    cur.execute(sql_email, (email, uid))
conn.commit()

#로그인


In [ ]:

sql_login = 'select uid, pwd from users where uid=%s and is_delited=0'

- case1 : uid가 없는경우


In [ ]:
#result가 NULL
uid = 'fool'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
result


- case2 : 비밀번호가 틀렸을 경우

In [ ]:
#둘다 맞는경우
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()

In [ ]:
db_pwd=result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else :
    print('incorrent password')

- case 3: OK

In [ ]:
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
db_pwd=result[1]
hashed_pwd = gen_pwd(pwd)
if db_pwd == hashed_pwd:
    print('Login')
else :
    print('incorrent password')

# 3가지 경우를 합쳐보자

In [ ]:
sql_login = 'select uid, pwd from users where uid=%s and is_delited=0'
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
if result:
    db_pwd=result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else :
        print('incorrent password')
else :
    print('BAD ID')

- 예외처리 할 경우

In [ ]:
sql_login = 'select uid, pwd from users where uid=%s and is_delited=0'
uid, pwd = 'djy', 'djy23'
cur.execute(sql_login, (uid,))
result = cur.fetchone()
try:
    db_pwd=result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('Login')
    else :
        print('incorrent password')
except :
    print('BAD ID')

In [30]:
cur.close()
conn.close()